# 📄 生成 AT 指令文档（支持多类型 + 校正优先 + 合并子节输出）

In [31]:
!pip install pandas jinja2 sphinx sphinx-rtd-theme

In [32]:
import os
import pandas as pd
import json
import re
from jinja2 import Environment, Template
from collections import defaultdict

In [33]:
# 自动创建 conf.py 和 CSS（如不存在）
conf_path = "docs/source/conf.py"
if not os.path.exists(conf_path):
    os.makedirs(os.path.dirname(conf_path), exist_ok=True)
    with open(conf_path, "w", encoding="utf-8") as f:
        f.write("""
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = []
templates_path = ['_templates']
exclude_patterns = []

html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']

def setup(app):
    app.add_css_file('custom.css')
""")
    print("✅ conf.py 已创建")
else:
    print("ℹ️ conf.py 已存在")

static_dir = 'docs/source/_static'
os.makedirs(static_dir, exist_ok=True)
with open(os.path.join(static_dir, 'custom.css'), 'w', encoding='utf-8') as f:
    f.write("body { font-family: 'Microsoft YaHei', sans-serif; }")
print("✅ CSS 准备完成")

ℹ️ conf.py 已存在
✅ CSS 准备完成


In [34]:
# 读取 CSV 和定义模板（含子类型合并输出）
CSV_PATH = 'at_commands_full_template.csv'  # 替换为你实际 CSV 文件名
OUTPUT_DIR = 'docs/source'

TEMPLATE_STRING = """
.. _cmd-{{ command_name | lower }}:

:ref:`{{ command_name }} <Basic-AT>`：{{ command_title }}
--------------------------------------------------------

{{ description or '' }}

{% for st in subtypes %}
{{ st.type }}命令
{{ (st.type + '命令') | length * '^' }}

**命令：**

::

    {{ st.fmt }}

**响应：**

::

{% for line in st.response.splitlines() %}
    {{ line }}
{% endfor %}

{% endfor %}
参数
^^^^
{% if parameters %}
{% for name, data in parameters.items() %}
- **{{ name }}**：
  {% if data.get('__desc__') %}
    {{ data['__desc__'] }}
  {% endif %}
  {% for key, value in data.items() if key != '__desc__' %}
    - {{ key }}：{{ value }}
  {% endfor %}
{% endfor %}
{% else %}无
{% endif %}

说明
^^^^
{{ note or '无' }}

示例命令
^^^^^^^^

::

{% for line in example.splitlines() %}    {{ line }}
{% endfor %}
"""

In [35]:
def sanitize_filename(fn):
    return re.sub(r'[<>:"/\\|?*\r\n]', '', fn)

def expand_multitype_row(row):
    types = [t.strip() for t in str(row.get('命令类型', '')).split(';')]
    cmds = [t.strip() for t in str(row.get('命令格式', '')).split(';')]

    # 响应校正优先
    raw_corr = str(row.get('响应校正', '')).strip().strip("'''")
    if raw_corr:
        resps = raw_corr.split(';')
    else:
        resps = [t.strip() for t in str(row.get('响应', '')).split(';')]

    examples = str(row.get('示例命令校正', '') or row.get('示例命令', '')).strip()

    max_len = max(len(types), len(cmds), len(resps), len(examples))
    expanded = []
    for i in range(max_len):
        expanded.append({
            '章节': row.get('章节'),
            '命令': row.get('命令'),
            '命令标题': row.get('命令标题'),
            '命令类型': types[i] if i < len(types) else '',
            '命令格式': cmds[i] if i < len(cmds) else '',
            '响应': resps[i] if i < len(resps) else '',
            '示例命令': examples[i] if i < len(examples) else '',
            '功能描述': row.get('功能描述'),
            '备注': row.get('备注'),
            '表数量': row.get('表数量'),
            '顺序': f"{row.get('顺序')}.{i+1}",
            '参数json': row.get('参数json')
        })
    return expanded

In [36]:
# 展开 CSV，然后渲染 .rst 文件
df0 = pd.read_csv(CSV_PATH, dtype=str).fillna("")
expanded = []
for _, row in df0.iterrows():
    expanded.extend(expand_multitype_row(row))
df = pd.DataFrame(expanded)
df.columns = df.columns.str.strip()

env = Environment()
template = env.from_string(TEMPLATE_STRING)

chapter_commands = {}
chapter_names = []

for chap, grp in df0.groupby('章节'):
    chap_name = str(chap).strip()
    chap_dir = os.path.join(OUTPUT_DIR, chap_name)
    os.makedirs(chap_dir, exist_ok=True)
    chapter_names.append(chap_name)

    # 用一个 dict 来记录哪些命令已经 add 过到 chapter_commands
    chapter_commands.setdefault(chap_name, [])

    for _, row in grp.iterrows():
        cmd = str(row['命令']).strip()

        # 如果还没加入命令名到目录，就加入一次
        if cmd not in chapter_commands[chap_name]:
            chapter_commands[chap_name].append(cmd)

        # 构建子类型列表
        subtypes = []
        # 假设你有 split(';') 逻辑
        types = [t.strip() for t in str(row.get('命令类型', '')).split(';')]
        fmts = [t.strip() for t in str(row.get('命令格式', '')).split(';')]
        # 响应校正优先
        raw_corr = str(row.get('响应校正', '')).strip().strip("'''")
        if raw_corr:
            resps = raw_corr.split(';')
        else:
            resps = [t.strip() for t in str(row.get('响应', '')).split(';')]
        examples = [t.strip() for t in str(row.get('示例命令', '')).split(';')]

        max_len = max(len(types), len(fmts), len(resps), len(examples))

        for i in range(max_len):
            st_type = types[i] if i < len(types) else ""
            st_fmt = fmts[i] if i < len(fmts) else ""
            st_resp = resps[i] if i < len(resps) else ""
            st_ex = examples[i] if i < len(examples) else ""
            subtypes.append({
                'type': st_type,
                'fmt': st_fmt,
                'response': st_resp,
                'examples': st_ex
            })

        # 从首行拿命令标题 / 描述 / 参数 / 备注 等
        row0 = grp.iloc[0]
        command_title = str(row0.get('命令标题', '')).strip()
        description = str(row0.get('功能描述', '')).strip()
        note = str(row0.get('备注', '')).strip()
        param_json = row0.get('参数json', '{}')
        # 参数解析同之前 logic
        try:
            parameters = json.loads(param_json)
        except:
            parameters = {}
        if isinstance(parameters, list):
            pdict = {}
            for p in parameters:
                name = p.get('name', '').strip()
                desc = p.get('desc', '').strip()
                valmap = p.get('valmap', {}) or {}
                pdict[name] = {'__desc__': desc}
                for k, v in valmap.items():
                    pdict[name][k] = v
            parameters = pdict

        # 渲染单个命令的 rst 内容 (含多个子类型)
        out_path = os.path.join(chap_dir, f"{cmd}.rst")
        content = template.render(
            command_name=cmd,
            command_title=command_title,
            description=description,
            note=note,
            parameters=parameters,
            subtypes=subtypes
        )
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(content)

print("✅ 所有 RST 文件已生成完毕")

UndefinedError: 'example' is undefined

In [ ]:
from jinja2 import Template

chapter_tmpl = """{{ chapter }}
{{ '=' * chapter|length }}

.. toctree::
   :maxdepth: 1

{% for c in cmds %}   {{ c }}
{% endfor %}
"""

main_tmpl = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapter_names %}   {{ ch }}/index
{% endfor %}
"""

for chap, cmds in chapter_commands.items():
    idx = os.path.join(OUTPUT_DIR, chap, 'index.rst')
    with open(idx, 'w', encoding='utf-8') as f:
        f.write(Template(chapter_tmpl).render(chapter=chap, cmds=cmds))

main_idx = os.path.join(OUTPUT_DIR, 'index.rst')
with open(main_idx, 'w', encoding='utf-8') as f:
    f.write(Template(main_tmpl).render(chapter_names=chapter_names))

print("✅ 所有章节 index.rst 已生成")

✅ 所有章节 index.rst 已生成


In [ ]:
!sphinx-build -b html docs/source docs/build/html -c docs/source
print("✅ HTML 构建完成，可查看 docs/build/html/index.html")

Running Sphinx v8.2.3
loading translations [en]... done
loading pickled environment... The configuration has changed (2 options: 'html_permalinks_icon', 'jquery_use_sri')
done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 3 source files that are out of date
updating environment: 0 added, 3 changed, 0 removed
reading sources... [ 33%] 2/AT+CSMS
reading sources... [ 67%] 2/index
reading sources... [100%] index

looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... 
copying static files... 
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\build\html\_static\basic.css
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\build\html\_static\documentation_options.js
Writ

C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\source\2\AT+CSMS.rst:11: WARNING: Title underline too short.

执行命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\source\2\AT+CSMS.rst:35: WARNING: Title underline too short.

查询命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\source\2\AT+CSMS.rst:35: WARNING: Title underline too short.

查询命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\source\2\AT+CSMS.rst:55: WARNING: Title underline too short.

测试命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html\docs\source\2\AT+CSMS.rst:55: WARNING: Title underline too short.

测试命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code